In [36]:
#cose da eseguire
#in un terminale (1) -> docker-compose up
#in un altro terminale (2) -> docker exec -it ollama bash
#nel terminale 2 -> ollama pull phi3


In [ ]:
!pip install elasticsearch tqdm openai

In [2]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [3]:
import json

#create the docs_raw variable from the .json file
with open('documents.json','rt') as f_in:
    docs_raw = json.load(f_in)

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)
        

In [4]:
def build_prompt(query,search_results):
    prompt_template="""
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't cointain the answer, output NONE.

    QUESTION: {question}

    CONTEXT:
    {context}
    """

    
    context = ""

    for doc in search_results:
        context= context + f"section: {doc['section']}\n\nquestion: {doc['question']}\n\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query,context=context)
    return prompt

In [5]:
def llm(prompt):

    response=client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content":prompt}]
    )
    return response.choices[0].message.content
    #import ollama
    #response = ollama.generate(model="gemma:2b", prompt=prompt)
    #return response

In [46]:
print(response['response'])

Yes, you can still enroll in the course after the start date. You will be able to submit homeworks and continue working on the final project at your own pace after it finishes.


In [6]:
query = "the course has already started, can I still enroll?"


In [8]:
def rag(query):
    results = search_elastic(query)
    prompt = build_prompt(query,results)
    response = llm(prompt)
    return response['response']

In [3]:
answer = rag(query)
print(answer)

NameError: name 'rag' is not defined

In [21]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay          32G   26G  3.8G  88% /
tmpfs            64M     0   64M   0% /dev
shm              64M     0   64M   0% /dev/shm
/dev/root        29G  9.3G   20G  33% /vscode
/dev/loop5       32G   26G  3.8G  88% /workspaces
/dev/sdb1       118G   32G   81G  28% /tmp


In [23]:
!docker ps


CONTAINER ID   IMAGE                                                 COMMAND                  CREATED      STATUS             PORTS                                                                                  NAMES
bee1db99429f   ollama/ollama                                         "/bin/ollama serve"      3 days ago   Up About an hour   0.0.0.0:11434->11434/tcp, :::11434->11434/tcp                                          ollama
1fbc4c376a5f   docker.elastic.co/elasticsearch/elasticsearch:8.4.3   "/bin/tini -- /usr/l…"   3 days ago   Up About an hour   0.0.0.0:9200->9200/tcp, :::9200->9200/tcp, 0.0.0.0:9300->9300/tcp, :::9300->9300/tcp   elasticsearch


In [14]:
!find /workspaces -type f -exec du -h {} + | sort -hr | head -n 20


644K	/workspaces/LLM-RAG/documents.json
612K	/workspaces/.codespaces/.persistedshare/vsserverterminallog.txt
472K	/workspaces/.codespaces/.persistedshare/vsserverhostlog.txt
352K	/workspaces/LLM-RAG/HUGGING_FACE_flan.ipynb
304K	/workspaces/.codespaces/shared/codespaceStatusTool.js
232K	/workspaces/LLM-RAG/.git/objects/pack/pack-26a05d4bb044a8d999de800b322d70bf8df23083.pack
200K	/workspaces/LLM-RAG/.ipynb_checkpoints/rag-intro2-checkpoint.ipynb
16K	/workspaces/LLM-RAG/rag-intro.ipynb
16K	/workspaces/.oryx/build.log
12K	/workspaces/LLM-RAG/rag-intro2.ipynb
8.0K	/workspaces/LLM-RAG/myollama.ipynb
8.0K	/workspaces/LLM-RAG/.git/objects/51/eba762f9584b199a3d89c05451a728ec62da71
8.0K	/workspaces/LLM-RAG/.git/hooks/pre-rebase.sample
8.0K	/workspaces/LLM-RAG/.git/hooks/fsmonitor-watchman.sample
8.0K	/workspaces/.codespaces/shared/read-config.json
8.0K	/workspaces/.codespaces/.persistedshare/devcontainers-cli/cache/env-loginInteractiveShell.json
8.0K	/workspaces/.codespaces/.persistedshare/creat

In [9]:


# Find large directories under /
!du -h /usr | sort -hr | head -n 20


8.2G	/usr
5.4G	/usr/local
1.2G	/usr/local/bin
1.1G	/usr/share
1.1G	/usr/local/python
1.1G	/usr/lib
895M	/usr/local/lib/ollama
895M	/usr/local/lib
844M	/usr/local/sdkman
820M	/usr/local/sdkman/candidates
726M	/usr/share/dotnet
668M	/usr/local/python/3.12.1
664M	/usr/local/sdkman/candidates/java
613M	/usr/local/python/3.12.1/lib
577M	/usr/lib/x86_64-linux-gnu
532M	/usr/local/python/3.12.1/lib/python3.12
464M	/usr/bin
419M	/usr/local/share
402M	/usr/local/share/nvm
401M	/usr/local/python/3.11.9
sort: write failed: 'standard output': Broken pipe
sort: write error


#elastic search

In [10]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')
index_name = "course-questions"
import tqdm
#dont run this cell several times, or (idk why) some doc duplicated. If so, try to delete the index ->
#es_client.indices.delete(index=index_name)
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)



100%|██████████████████████████████████████████████████████████████████████| 948/948 [00:31<00:00, 30.42it/s]


In [ ]:
#CREATING INDICES
#they are saved permanently, so you have to run it only 1 time, no need to rerun after power-up

index_settings = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
            "properties": {
                "text": {"type": "text"},
                "section": {"type": "text"},
                "question": {"type": "text"},
                "course": {"type": "keyword"} 
            }
        }
    }
    

    
es_client.indices.create(index=index_name, body=index_settings)

In [12]:
#"filter" chooose the document that match with the "course"
#"question^3" means that the field is 3 times more important than the other
#"size" is the number of document retrieve


def search_elastic(query):
    search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    results_doc = []
    
    for hit in response['hits']['hits']:
        results_doc.append(hit['_source'])
    
    return results_doc

In [2]:
!unistall

/bin/bash: unistall: command not found
